# To Do (development notes only)
# REMOVE LATER BEFORE SUBMITTING
### Feature Engineering
* Other model experimentation / optimization
    - (model examples including nonlinear)
    - K Nearest Neighbors
    - Naive Bayes
        * Multinomial
        * Gaussian
        * Bernoulli
    - GMM
    - SVM
* Discuss pros/cons of each of the models compared to results


### Model Selection
Test with models that can support predict_proba to predict probability of all categories

# W207 Final Project
### Kaggle Competition
[San Francisco Crime Statistics](https://www.kaggle.com/c/sf-crime)
  
###  Team Members
Chuck Bolin, Matthew Burke, Yun-Hui Fan

In [11]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import utils

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA

### Data Loading
Load in training and test data to variables

In [3]:
train_all = pd.read_csv('../data/train.csv', delimiter=',', parse_dates=['Dates'])
# test_final contains data which will be submitted to kaggle after predicting categories
# Includes ID field not found in training data
# Does not include description, category or resolution fields in training data
test_final = pd.read_csv('../data/test.csv', delimiter=',')

## Feature Engineering

### Date Features
Take the single time to create features for Year, Month, Day and Hour

In [4]:
def dateAttributes(df):
    df = df.copy()
    df['Year'] = pd.DatetimeIndex(df['Dates']).year
    df['Month'] = pd.DatetimeIndex(df['Dates']).month
    df['Day'] = pd.DatetimeIndex(df['Dates']).day
    df['Hour'] = pd.DatetimeIndex(df['Dates']).hour
    return df

# Extract elements of dates
train_all = dateAttributes(train_all)

# Column names to binarize:
bin_cols = ['Year', 'Month', 'Hour', 'DayOfWeek', 'PdDistrict']

# Binarize columns identified above and add to dataframe
for column in bin_cols:
    dummies = pd.get_dummies(train_all[column])
    train_all[dummies.columns] = dummies

# Encode all categories into integers for use later
le = LabelEncoder()
labels_all = le.fit_transform(train_all['Category'])

    
#Extract categories names from encoder for use later
categories = le.classes_

### XY Coordinates
Round X and Y features to 2 decimal places, convert to strings and then combine into single value
Binarize new feature and concatenate to training data

In [5]:
fun = lambda x: str(round(x, 2))
temp = train_all['X'].map(fun) + train_all['Y'].map(fun)
lb = LabelBinarizer()
train_all = np.concatenate((train_all, lb.fit_transform(temp)), axis=1)

### Mapping Addresses
Extract street names from `Address` column and combine if multiple street names, i.e. for a city block.
Ultimately creates 12000+ features, which isn't necessarily more helpful than XY coordinates, especially since we are removing the block numbers which reduces precision even further.

In [33]:
add = train_all[:,6]
streets = []
for i in range(0,len(add)):
    matches = re.findall('[A-Z]{3,}|[0-9]+TH',add[i])
    if len(matches)==1:
        streets.append(matches[0])
    else:
        streets.append(''.join(matches))
print 'Total Street Name Features:', len(np.unique(streets))

Total Street Name Features: 12758


## Split Data Into Train/Test
Designate 1/3 of the data to be test data and the other 2/3 to be training data for development purposes

In [6]:
# Define fraction of data to be used as test data
fraction = 0.33

# Split data into training and test data/labels randomly according to fraction specified
train_labels, test_labels, train_data, test_data = train_test_split(labels_all, train_all, test_size=fraction)


print 'Check that data has been formed correctly:\n'

print 'Training data shape: ', train_data.shape
print 'Training labels shape: ', len(train_labels)#train_labels.shape

print 'Test data shape: ', test_data.shape
print 'Test labels shape: ', len(test_labels)#test_labels.shape

print('Train labels: ', len(np.unique(train_labels)))
print('Test  labels: ', len(np.unique(test_labels)))

print '\nTop row of training data:\n'

print train_data [0,]

Check that data has been formed correctly:

Training data shape:  (588292, 206)
Training labels shape:  588292
Test data shape:  (289757, 206)
Test labels shape:  289757
('Train labels: ', 39)
('Test  labels: ', 39)

Top row of training data:

[Timestamp('2006-02-07 07:45:00') 'VEHICLE THEFT' 'STOLEN TRUCK' 'Tuesday'
 'BAYVIEW' 'NONE' '1400 Block of WAYLAND ST' -122.41645145514299
 37.7230044830156 2006 2 7 7 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]


### PCA
Performing PCA on binarized Year, Month, Hour, DayOfWeek, District and XY coordinate

In [30]:
pca_mod = PCA(n_components=120).fit(train_all[:200000,12:])
for i in range(5,120, 5):
    var = sum(pca_mod.explained_variance_ratio_[range(1,i)])
    print 'Components: ', '{:2}'.format(i), '    Variance Explained: ', var

Components:   5     Variance Explained:  0.019597141065
Components:  10     Variance Explained:  0.0346923201475
Components:  15     Variance Explained:  0.0478148962346
Components:  20     Variance Explained:  0.0558327942307
Components:  25     Variance Explained:  0.0620686855089
Components:  30     Variance Explained:  0.067480872356
Components:  35     Variance Explained:  0.072116252412
Components:  40     Variance Explained:  0.0757860630109
Components:  45     Variance Explained:  0.0785184129746
Components:  50     Variance Explained:  0.0803437074549
Components:  55     Variance Explained:  0.0818741339834
Components:  60     Variance Explained:  0.0830849656831
Components:  65     Variance Explained:  0.084168500895
Components:  70     Variance Explained:  0.0850772499066
Components:  75     Variance Explained:  0.0858472412636
Components:  80     Variance Explained:  0.0864942576853
Components:  85     Variance Explained:  0.0870557806639
Components:  90     Variance Explai

## Model Evaluation
We define subsets of the training and test data to use for development. Larger datasets will be used for full validation

In [7]:
# Columns to use in baseline classification
cols = range(13, 206)

dev_data = train_data#[0:100000,]
dev_labels = train_labels#[0:70000]
t
dev_test_data = test_data#[0:30000,]
dev_test_labels = test_labels#[0:30000]

print('Train labels: ', len(np.unique(dev_labels)))
print('Test  labels: ', len(np.unique(dev_test_labels)))

('Train labels: ', 39)
('Test  labels: ', 39)


### Logistic Regression
We tested logistic regression as a simple, baseline model since it is intuitive and the prediction factors are easily identifiable. First we use a gridsearch on C values to find optimal parameters.

In [47]:
c_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
params = {'C': c_values}
grid = GridSearchCV(estimator=LogisticRegression(), param_grid=params, scoring='neg_log_loss')
grid.fit(dev_data[:,cols], dev_labels)
print grid
print grid.best_params_

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)
{'C': 0.3}


In [106]:
lgr = LogisticRegression(C=0.3)
print 'Begin training'
lgr.fit(dev_data[:,cols], dev_labels)
print 'Completed training'

print 'Begin prediction'
pred_probs = lgr.predict_proba(dev_test_data[:,cols])

print 'Completed prediction\n'

print 'Log score: ', metrics.log_loss(dev_test_labels, pred_probs, labels=range(0,39)), '\n'

print 'Probability prediction examples: ', pred_probs[0,]

Begin training
Completed training
Begin prediction
Completed prediction

Log score:  2.50419479283 

Probability prediction examples:  [  2.05772035e-03   9.64082077e-02   1.41803703e-04   5.20219719e-04
   2.23870262e-02   1.08380566e-02   3.00517150e-03   3.60632917e-02
   6.72280000e-03   3.22072579e-04   2.36610824e-04   5.36850724e-03
   3.06558696e-03   4.96320498e-03   1.78035746e-04   1.67127343e-03
   1.13397203e-01   2.13864695e-03   7.14063657e-04   6.93625772e-02
   7.59141540e-02   1.19626048e-01   6.56260835e-05   5.85852760e-02
   1.38840981e-04   2.13658750e-02   1.54885118e-02   7.72431219e-03
   5.38876447e-03   5.20608425e-04   2.63852127e-03   8.80772743e-04
   2.31179088e-02   4.62691085e-05   1.03402113e-02   5.53862793e-02
   1.67644117e-01   4.35304298e-02   1.20353940e-02]


### K Nearest Neighbors
The K nearest neighbors classifier uses clustering in order to group features together. 
It seems appropriate that we use this algorithm, considering that we are dealing with actual neighborhoods in the data. We need to first find the optimal number of clusters to use.
Additionally, we will change the parameters to include the actual X/Y coordinate values instead of the combined string proxy version since it can use the numerical data to form actual clusters in space.

In [ ]:
knnCols = range(7,13)

n_values = [200, 300, 500]
params = {'n_neighbors': n_values}
grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params, scoring='neg_log_loss')
grid.fit(dev_data[:,knnCols], dev_labels)
print grid
print grid.get_params(0)
print grid.best_params_

In [12]:
knnCols = range(7,13)
knn = KNeighborsClassifier(n_neighbors=500)
print('Begin fitting')
knn.fit(dev_data[:,knnCols], dev_labels)
print('Completed fitting')
print('Begin prediction')
predicted = knn.predict_proba(dev_test_data[:,knnCols])
print('Completed prediction')
print metrics.log_loss(dev_test_labels, predicted, labels=range(0,39))

Begin fitting
Completed fitting
Begin prediction
Completed prediction
2.86875081942


### Random Forest

In [20]:
rfc = RandomForestClassifier(n_estimators=50)
print('Begin fitting')
rfc.fit(dev_data[:,knnCols],dev_labels)
print('Completed fitting')
print('Begin prediction')
predicted = rfc.predict_proba(dev_test_data[:,knnCols])
print('Completed prediction')
print metrics.log_loss(dev_test_labels, predicted, labels=range(0,39))

Begin fitting
Completed fitting
Begin prediction
Completed prediction
7.00851969519


### Naive Bayes
#### Bernoulli
Bernoulli Naive Bayes is suited towards discrete data, specifically binary. 
The data for this problem is binary in a category vs all so it may be a good fit.

In [63]:
### Perform BernoulliNB analysis here including parameter optimization

#### Multinomial
Multi-class options may be better fit for data than Binomial
Use label encoded data as opposed to binary?

In [65]:
### Perform MultinomialNB analysis here including parameter optimizaiton of alpha

#### Gaussian
Gaussian models for each of the classes? Does this even make sense, given that there are so many categories?
Will models with too few skew the data incorrectly? Discuss this

In [67]:
### Perform GaussianNB analysis here for all categories,
### predict based on log odds for each of the individual category classifiers

### SVM

In [ ]:
### Perform SVM analysis here

## Final Model Predictions

In [ ]:
#test_final = dateAttributes(test_final)

In [94]:
# Binarize columns in initial data processing above and add to dataframe
for column in bin_cols:
    print 'Starting ', column
    dummies = pd.get_dummies(test_final[column])
    test_final[dummies.columns] = dummies
    print 'Finished ', column

# Copy IDs fr each row in final test data just in case...
ids = test_final['Id']

# Convert final data data intl numpy object for processing
test_final_data = np.array(test_final)

Starting  Year
Finished  Year
Starting  Month
Finished  Month
Starting  Hour
Finished  Hour
Starting  DayOfWeek
Finished  DayOfWeek
Starting  PdDistrict
Finished  PdDistrict


In [97]:
print test_final_data[0,range(12,len(test_final.columns))]

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [98]:
# Train final model on all training data available
lgrFinal = LogisticRegression()

print 'Beginning training'
lgrFinal.fit(train_data_all[:,cols], labels_all)
print 'Completed training'

Beginning training
Completed training


In [101]:
# Calculate predicted probabilities for each category for final test data

print 'Begin prediction'
pred_probs = lgrFinal.predict_proba(test_final_data[:,range(11,len(test_final.columns))])
print 'Completed prediction'

Begin prediction
Completed training


NameError: name 'cat' is not defined

In [116]:
# Output predictions to csv file for kaggle submission
output = pd.DataFrame(pred_probs, columns=categories)
file_name = '../data/matthew_submission.csv'
print 'Output to file: ', file_name
output.to_csv(file_name, index=True, index_label='Id')
print 'File creation complete'

Output to file:  ../data/matthew_submission.csv
File creation complete


In [118]:
print output.shape

(884262, 39)


In [17]:
print len(np.unique(train_all['Address']))

23228
